In [113]:
import pandas as pd 
import numpy as np

## 1 Exercise: Business cycle correlations

#### 1.1

In [114]:
import glob

data = []

DATA_PATH = '/Users/andreas/NHH/tech2-local/TECH2-H24/data/FRED'

files = glob.glob(f'{DATA_PATH}/FRED_monthly_*.csv')

for file in files:
    data.append(pd.read_csv(file, parse_dates=['DATE']))

#### 1.2

In [115]:
df = pd.concat(data, axis = 0).set_index('DATE')
df.head()

,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART,INFLATION
DATE,,,,,,
1950-01-01,23.5,6.5,NaN,NaN,58.9,NaN
1950-02-01,23.6,6.4,NaN,NaN,58.9,NaN
1950-03-01,23.6,6.3,NaN,NaN,58.8,NaN
1950-04-01,23.6,5.8,NaN,NaN,59.2,NaN
1950-05-01,23.8,5.5,NaN,NaN,59.1,NaN


#### 1.3

In [116]:
import os.path

fn = os.path.join(DATA_PATH, 'GDP.csv')
GDP = pd.read_csv(fn, parse_dates=['DATE'], index_col=['DATE'])

df = df.merge(GDP, on='DATE', how='inner')
df.head()

,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART,INFLATION,GDP
DATE,,,,,,,
1950-01-01,23.5,6.5,NaN,NaN,58.9,NaN,2346.1
1950-04-01,23.6,5.8,NaN,NaN,59.2,NaN,2417.7
1950-07-01,24.1,5.0,NaN,NaN,59.1,NaN,2511.1
1950-10-01,24.5,4.2,NaN,NaN,59.4,NaN,2559.2
1951-01-01,25.4,3.7,NaN,NaN,59.1,NaN,2594.0


#### 1.4

In [117]:
df_changes = df[['CPI', 'GDP']].pct_change()*100

variables = ['UNRATE', 'FEDFUNDS', 'REALRATE', 'LFPART']

df_changes[variables] = df[variables].diff()

df_changes.head()

,CPI,GDP,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,,
1950-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1950-04-01,0.425532,3.051873,-0.7,NaN,NaN,0.3
1950-07-01,2.118644,3.863176,-0.8,NaN,NaN,-0.1
1950-10-01,1.659751,1.915495,-0.8,NaN,NaN,0.3
1951-01-01,3.673469,1.359800,-0.5,NaN,NaN,-0.3


In [119]:
df_changes.corr().loc['GDP']

CPI         0.970083
GDP         1.000000
UNRATE     -0.158591
FEDFUNDS   -0.008052
REALRATE   -0.137800
LFPART      0.803695
Name: GDP, dtype: float64